# CV part one

В этой тетрадке мы рассмотрим задачу распознавания лиц на примере датасета [CelebA](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html)

**Предполагаем, что ноутбук запущен внутри Yandex DataSphere**

In [1135]:
#!bash
rm -r .\tboard_logs

In [1005]:
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2
import os
from collections import defaultdict

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader, Sampler
from torchvision.models import resnet34
from torch.utils.tensorboard import SummaryWriter

from warnings import filterwarnings

filterwarnings("ignore")

## Data

Качаем архив с данными с Yandex Object Storage и распаковываем в текущую папку.

Структура архива:
- /celeba_data/
    - train.csv
    - val.csv
    - images/{image}.jpg

CSV файлы содержат название файла (`image`) и его лейбл (`label`).

In [2]:
from cloud_ml.storage.api import Storage

s3 = Storage.s3(access_key="Le9tg70HQEJsoGqjqXH8", secret_key="NV75mCPkC0PEd35ImyDI5vI7p40YGFOYZgkH7moa")
# downloading contents of the remote file into the local one
s3.get('dl-hse-2021/celeba_data.zip', './celeba_data.zip')

In [4]:
#!:bash
unzip -q ./celeba_data.zip -d ./ && rm celeba_data.zip

## Задание 1
**(0.2 балла)** Напишите класс датасет, который будет возвращать картинку и ее лейбл.

In [2]:
class CelebADataset(Dataset):
    def __init__(self, train=True):
        self._path = "celeba_data/"
        self.images_dir_path = self._path + "images/"
        self.file = self._path + "train.csv" if train else self._path + "val.csv"
        self.header = pd.read_csv(self.file)

    def __len__(self):
        return len(self.header)
    
    def __getitem__(self, index):
        img_name, label = self.header.iloc[index, :]
        
        img_path = Path(self.images_dir_path, img_name)
        img = self._read_img(img_path)
        
        return dict(sample=img, label=label)
        
    @staticmethod
    def _read_img(img_path: Path):
        img = cv2.imread(str(img_path.resolve()))
        img = img.astype(np.float32)
        img = np.transpose(img, (2, 0, 1)) / 255.
        
        return img

## Задание 2
**(0.2 балла)** Напишите функцию, которая будет считать метрику top-n accuracy.

$$TopN \ Accuracy = \frac{Number \ of \ objects \ with \ correct \ answer \ among \ topN \ predictions}{Total \ number \ of \ objects}$$

*Example:*

![image](https://www.baeldung.com/wp-content/ql-cache/quicklatex.com-ae746981c7a437b7e1fc2831e5d76d57_l3.svg)  
$Top3 \ Accuracy = \frac{4}{5} = 0.8$

*Hint:* Для каждого объекта выбираем `n` наиболее уверенных предсказаний. Если среди них есть правильный ответ, то увеличиваем числитель и знаменатель на единицу, иначе увеличиваем только знаменатель.

In [3]:
def top_n_accuracy(preds: np.ndarray,
                   targets: np.ndarray,
                   n_size: int) -> float:
    """
    Предполагается, что на preds приходит на вход в порядке убывания уверенности предсказания
    Так же предполагаем, что уже убраны дубликаты
    """
    return (targets.reshape(-1, 1) == preds[:, :n_size]).any(axis=1).mean()

In [4]:
targets = np.array([1, 0, 2, 2])
preds = np.array([
    [1, 0, 3],
    [3, 4, 0],
    [0, 2, 3],
    [3, 4, 2]
])

assert top_n_accuracy(preds, targets, 2) == 2 / 4

## Задание 3
**(0.2 балла)** Решите задачу без дообучения.

*Step-by-step:*
1. Инициализируйте предобученную сетку (`backbone`).
1. Прогоните через нее все картинки из валидационного датасета и сложите полученные эмбеддинги в массив.
1. Для каждого вектора найдите ближайшие к нему векторы и отсортируйте их по расстоянию (cosine, euclidian, ...). Лейблы соседних векторов будут предсказаниями для текущего вектора.
1. Оставьте топ-5 уникальных предсказаний.
1. Посчитайте и выведите метрики:
    1. top-1 accuracy
    1. top-5 accuracy

*Вопросы:*
1. Зачем мы заменяем последний линейный слой на `Identity` ?
1. Зачем используем на сетке метод `eval` ?

*Hints:*
1. Для расчета попарных расстояний лучше не использовать циклы, а считать все в матрицах. Описание подхода к расчету L2 расстояний: [link](https://math.stackexchange.com/questions/3147549/compute-the-pairwise-euclidean-distance-matrix)
1. Так можно использовать sklearn реализации: [link](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics.pairwise)
1. Для получения top-k предсказаний не обязательно сортировать весь массив.

*Ответы:*

1. Identity - пропускает выходы последнего слоя as is. В этой задаче не нужна голова-классификатор, достаточно будет скоров/эмбедингов бэкбона
1. Некоторые блоки сети по разному ведут себя на трейне и инференсе, методом `eval()` мы переключаем их на режим инференса

In [5]:
#!L

DEVICE = torch.cuda.current_device() if torch.cuda.is_available() else "cpu"

backbone = resnet34(pretrained=True)
backbone.fc = nn.Identity()
backbone = backbone.eval()
backbone.to(DEVICE)

celeb_val = DataLoader(CelebADataset(train=False), shuffle=False, pin_memory=True)

In [6]:
#!L

@torch.no_grad()
def get_embeds(model: nn.Module, dataloader: DataLoader) -> np.ndarray:
    embeddings = []
    labels = []
    model.eval()
    for batch in tqdm(dataloader):
        embedding = model.forward(batch["sample"].to(DEVICE))
        embeddings.append(embedding.cpu().numpy())
        labels.append(batch["label"].cpu().numpy())
        torch.cuda.empty_cache()
    
    return np.concatenate(embeddings), np.concatenate(labels)


def l2_dist(x: np.ndarray) -> np.ndarray:
    # своровано с лекции
    a = x.dot(x.T)
    b = np.diag(a)
    dist = np.sqrt(b.reshape(-1, 1) - 2 * a + b)
    return dist

embeds, labels = get_embeds(backbone, celeb_val)

dists = l2_dist(embeds)

np.save("val_embeds.npy", embeds)
np.save("val_labels.npy", labels)
np.save("val_dists.npy", dists)

100%|██████████| 19867/19867 [04:59<00:00, 66.25it/s]


In [13]:
#!M
# embeds = np.load("embeds.npy")
# dists  = np.load("dists.npy")
def top_n_preds(labels, dists, n):
    """ Отбор top-n уникальных предсказаний"""
    sorted_preds = labels[np.argsort(dists, axis=1)][:, 1:]
    # https://stackoverflow.com/questions/12926898/numpy-unique-without-sort
    top_n_preds = []
    for row in sorted_preds:
        indexes = np.unique(row, return_index=True)[1]
        preds = [row[index] for index in sorted(indexes)]
        top_n_preds.append(preds[:n])
    return np.array(top_n_preds)

def compute_accs(labels, dists, n=[1, 5]):
    top_preds = top_n_preds(labels, dists, max(n))
    accs = [top_n_accuracy(top_preds, labels, n_) for n_ in n]
    return accs
    
baseline_top_1_acc, baseline_top_5_acc = compute_accs(labels, dists)

print(f"TOP 1 ACCURACY: %.3f" % baseline_top_1_acc)
print(f"TOP 5 ACCURACY: %.3f" % baseline_top_5_acc)

TOP 1 ACCURACY: 0.160
TOP 5 ACCURACY: 0.273


## Задание 4
**(0.4 балла)** Решите задачу с дообучением на эмбеддингах.

*Step-by-step:*
1. Напишите небольшую сетку произвольной архитектуры, которая будет использовать эмбеды, выдаваемые `backbone` сетью.
1. Напишите класс Dataset, который будет возвращать эмбединг и лейбл.
1. Напишите класс Sampler [PyTroch docs](https://pytorch.org/docs/stable/data.html#data-loading-order-and-sampler), который будет отвечать за правильность сбора тренировочных батчей: якорный пример, позитивный, негативный.
1. Обучите ее на тренировочном датасете:
    1. Лосс -- [triplet loss](https://pytorch.org/docs/stable/generated/torch.nn.TripletMarginLoss.html).
    1. Метрика -- top-5 accuracy.
1. Посчитайте top-1 и top-5 accuracy на валидации. Насколько сильно они отличаются от того, что получилось в предыдущем задании?


*Hints:*
1. Убедитесь, что у каждого лейбла есть как минимум 2 примера, иначе не получится достать позитивный пример.
1. Лучше предварительно прогнать все картинки из трейна и сохранить полученные эмбеддинги, чтобы при обучении сети грузить только эмбеды (векторы).

In [14]:
#!L
# сразу доформируем эмбеды

celeb_train = DataLoader(CelebADataset(train=True), shuffle=False, pin_memory=True)

train_embeds, train_labels = get_embeds(backbone, celeb_train)

np.save("train_embeds.npy", train_embeds)
np.save("train_labels.npy", train_labels)

100%|██████████| 162770/162770 [40:17<00:00, 67.32it/s]


In [1078]:
#!L

class SimpleDataset(Dataset):
    def __init__(self, embeddings: torch.tensor,
                 labels: torch.tensor):

        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        if isinstance(idx, int):
            sample = self.embeddings[idx]
            label = self.labels[idx]
            
            return dict(
                sample=sample,
                label=label
            )
        else:
            assert len(idx) == 3
            
            anc_idx, pos_idx, neg_idx = idx
            
            samples = self.embeddings[anc_idx], self.embeddings[pos_idx], self.embeddings[neg_idx]
            labels  = self.labels[anc_idx],     self.labels[pos_idx],     self.labels[neg_idx]

            return dict(
                samples=samples,
                labels=labels
            )


class SimpleTripletSampler(Sampler):
    def __init__(self, dataset: Dataset):
        super().__init__(dataset)

        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __iter__(self):
        for anchor_idx in range(len(self.dataset)):
            positive_idx = self._mine_positive(anchor_idx)
            negative_idx = self._mine_negative(anchor_idx)

            yield anchor_idx, positive_idx, negative_idx

    def _mine_positive(self, anchor_idx: int):

        anchor_label = self.dataset.labels[anchor_idx]
        pos_idxs = torch.nonzero(self.dataset.labels == anchor_label)
        pos_idx = pos_idxs[np.random.randint(low=0, high=pos_idxs.shape[0])]
        
        if len(pos_idxs) == 1:
            return anchor_idx

        return pos_idx.squeeze()

    def _mine_negative(self, anchor_idx: int):

        anchor_label = self.dataset.labels[anchor_idx]
        neg_idxs = torch.nonzero(self.dataset.labels != anchor_label)
        neg_idx = neg_idxs[np.random.randint(low=0, high=neg_idxs.shape[0])]

        return neg_idx.squeeze()

# sanity check
def sanity():
    train_embdes = torch.tensor(np.load("train_embeds.npy"), device=DEVICE)
    train_labels = torch.tensor(np.load("train_labels.npy"), device=DEVICE)
    dataset = SimpleDataset(train_embeds, train_labels)
    sampler = SimpleTripletSampler(dataset)
    loader  = DataLoader(dataset=dataset, sampler=sampler, batch_size=10)
    
    batch = next(iter(loader))
    print(batch["samples"][0])
    print(batch["samples"][1])
    print(batch["samples"][2])
    
    assert (batch["labels"][0] == batch["labels"][1]).all(), 'Positive labels dont match'
    assert (batch["labels"][0] != batch["labels"][2]).all(), 'Negative labels dont mismatch'
    print("Success!")
    
sanity()

tensor([[1.8756e+00, 4.5872e-01, 5.4045e-01,  ..., 1.0325e+00, 1.3787e+00,
         4.7378e-01],
        [1.1979e+00, 4.4343e-01, 3.5210e-04,  ..., 5.4469e-01, 1.6519e-01,
         3.5330e-01],
        [1.4011e+00, 3.4878e-01, 5.1260e-01,  ..., 1.3276e+00, 3.9931e-01,
         1.5838e+00],
        ...,
        [1.2097e+00, 1.2873e+00, 8.8493e-01,  ..., 8.3415e-01, 9.1353e-02,
         5.1008e-01],
        [1.2309e+00, 2.1698e-01, 0.0000e+00,  ..., 5.6329e-01, 5.8289e-01,
         1.2876e+00],
        [1.0101e+00, 6.1693e-01, 8.3480e-01,  ..., 1.1601e+00, 3.7407e-01,
         1.0441e+00]])
tensor([[1.2422, 0.6934, 0.2338,  ..., 1.4113, 0.0898, 1.0202],
        [1.5253, 0.2900, 0.7709,  ..., 2.3567, 0.4107, 0.8077],
        [1.4011, 0.3488, 0.5126,  ..., 1.3276, 0.3993, 1.5838],
        ...,
        [0.5586, 0.3096, 0.5893,  ..., 0.8518, 1.5713, 1.4221],
        [1.2309, 0.2170, 0.0000,  ..., 0.5633, 0.5829, 1.2876],
        [1.0467, 0.6908, 0.2539,  ..., 1.6227, 0.2793, 0.7280]])
tensor

In [1125]:
#!L

class SomeModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
                        nn.Linear(in_features=512, out_features=1024),
                        nn.ReLU(),
                        nn.Linear(in_features=1024, out_features=1024),
                        nn.ReLU(),
                        nn.Linear(in_features=1024, out_features=512),
                    )
        self.triplet_loss = nn.TripletMarginLoss(margin=1.0, p=2)
    
    def forward(self, triplet):
#         print(self.triplet_loss(*triplet))
        anchor   = self.net(triplet[0])
        positive = self.net(triplet[1])
        negative = self.net(triplet[2])
#         print("3 ", anchor, positive, negative)
        return anchor, positive, negative
        
    
    def compute_all(self, triplet, labels):
        # computes batch-wise loss and accuracy
        loss = self.triplet_loss(*triplet)
        
        all_embeds = torch.cat(triplet).cpu().detach().numpy()
        all_labels = torch.cat(labels).cpu().detach().numpy()
        
        dists = l2_dist(all_embeds)
#         print("0 ", len(labels[0]), all_labels.shape, all_embeds.shape, triplet[0].shape)
        accs = compute_accs(all_labels, dists)
        
        return loss, dict(acc_1=accs[0], acc_5=accs[1])

In [1126]:
#!L

def set_seed(seed):
    # https://stackoverflow.com/questions/56354461/reproducibility-and-performance-in-pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)    

set_seed(12)


class Trainer:
    def __init__(self, model: nn.Module,
                 optimizer,
                 train_dataset: Dataset,
                 val_dataset: Dataset,
                 tboard_log_dir: str = "./tboard_logs/",
                 batch_size: int = 128,
                 n_hardest: int = 256):
        self.model = model
        self.optimizer = optimizer
#         self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3)
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.batch_size = batch_size
        self.train_sampler = SimpleTripletSampler(train_dataset)
        self.val_sampler = SimpleTripletSampler(val_dataset)
        self.hard_selector = nn.TripletMarginLoss(margin=1.0, p=2, reduction='none')
        self.n_hardest = n_hardest
        self.device = 'cpu'
        if torch.cuda.is_available():
            self.device = torch.cuda.current_device()
            self.model = self.model.to(self.device)

        self.global_step = 0
        self.train_writer = SummaryWriter(log_dir=tboard_log_dir + "train/")
        self.val_writer = SummaryWriter(log_dir=tboard_log_dir + "val/")
        
        
    def save_checkpoint(self, path):
        torch.save(self.model.state_dict(), path)
        
    @torch.no_grad()
    def select_hardest(self, triplet):
#         print("1 ", triplet)
        hard_idxs = self.hard_selector(*triplet)
#         print("2 ", hard_idxs)
        hard_idxs = list(hard_idxs.topk(self.n_hardest).indices.detach().cpu().numpy())
        
        return hard_idxs

    def train(self, num_epochs: int):
        model = self.model
        optimizer = self.optimizer

        train_loader = DataLoader(self.train_dataset, sampler=self.train_sampler, batch_size=self.batch_size)
        val_loader = DataLoader(self.val_dataset, sampler=self.val_sampler, batch_size=self.batch_size)
        best_loss = float('inf')

        for epoch in range(num_epochs):
            model.train()
            train_losses = []
            for batch in tqdm(train_loader):
#                 print(batch["samples"][0].shape)
                batch = {k: [v_.to(self.device) for v_ in v] for k, v in batch.items()}
                samples, labels = batch["samples"], batch["labels"]
#                 print("-1 ", batch["samples"][0].shape, batch["labels"][0].shape)
                samples = model.forward(samples)
                
                hardest_idxs = self.select_hardest(samples)
                labels = labels[0][hardest_idxs], labels[1][hardest_idxs], labels[2][hardest_idxs]
                samples = samples[0][hardest_idxs], samples[1][hardest_idxs], samples[2][hardest_idxs]

                loss, details = model.compute_all(samples, labels)
                train_losses.append(loss.item())
                
                for k, v in details.items():
                    self.train_writer.add_scalar(k, v, global_step=self.global_step)
                self.global_step += 1

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
            mean_train_loss = np.mean(train_losses)

            model.eval()
            val_losses = []
            val_logs = defaultdict(list)
            for batch in tqdm(val_loader):
                batch = {k: [v_.to(self.device) for v_ in v] for k, v in batch.items()}
                
                samples, labels = batch["samples"], batch["labels"]
                samples = model.forward(samples)
                
                loss, details = model.compute_all(samples, labels)
                val_losses.append(loss.item())
                for k, v in details.items():
                    val_logs[k].append(v)
                    
            val_logs = {k: np.mean(v) for k, v in val_logs.items()}
            for k, v in val_logs.items():
                self.val_writer.add_scalar(k, v, global_step=self.global_step)

            val_loss = np.mean(val_losses)
            
            if val_loss < best_loss:
                self.save_checkpoint("./best_checkpoint.pth")
                best_loss = val_loss
            
            print("Batch mean train Loss:  %.4f\tBatch mean val Loss:  %.4f" % (mean_train_loss, val_loss))
            
#             self.scheduler.step(val_loss)

In [1127]:
#!L

train_set = SimpleDataset(
                    torch.tensor(np.load("train_embeds.npy"), device=DEVICE, requires_grad=True), 
                    torch.tensor(np.load("train_labels.npy"), device=DEVICE, requires_grad=False)
            )
val_set   = SimpleDataset(
                    torch.tensor(np.load("val_embeds.npy"), device=DEVICE, requires_grad=True), 
                    torch.tensor(np.load("val_labels.npy"), device=DEVICE, requires_grad=False)
            )

model = SomeModel()
opt = optim.Adam(model.parameters(), lr=1e-4)
trainer = Trainer(model, opt, train_set, val_set, batch_size=1024, n_hardest=700)

trainer.train(20)

@torch.no_grad()
def em():
    embeddings = []
    labels = []
    model.eval()
    for batch in tqdm(DataLoader(val_set, shuffle=False)):
        embedding = model.net(batch["sample"].to(DEVICE))
        embeddings.append(embedding.cpu().numpy())
        labels.append(batch["label"].cpu().numpy())
        torch.cuda.empty_cache()
    
    return np.concatenate(embeddings), np.concatenate(labels)


new_embeds, new_labels = em()
new_dists = l2_dist(new_embeds)
top_1_acc, top_5_acc = compute_accs(new_labels, new_dists)

print(f"TOP 1 ACCURACY: %.3f" % top_1_acc)
print(f"TOP 5 ACCURACY: %.3f" % top_5_acc)

Batch mean train Loss:  0.6256	Batch mean val Loss:  0.3732
Batch mean train Loss:  0.5260	Batch mean val Loss:  0.3403
Batch mean train Loss:  0.5044	Batch mean val Loss:  0.3268
Batch mean train Loss:  0.4856	Batch mean val Loss:  0.3171
Batch mean train Loss:  0.4721	Batch mean val Loss:  0.3229
Batch mean train Loss:  0.4644	Batch mean val Loss:  0.3170
Batch mean train Loss:  0.4530	Batch mean val Loss:  0.3070
Batch mean train Loss:  0.4447	Batch mean val Loss:  0.3086
Batch mean train Loss:  0.4408	Batch mean val Loss:  0.3077
Batch mean train Loss:  0.4359	Batch mean val Loss:  0.3030
Batch mean train Loss:  0.4306	Batch mean val Loss:  0.3002
Batch mean train Loss:  0.4290	Batch mean val Loss:  0.2930
Batch mean train Loss:  0.4246	Batch mean val Loss:  0.3002
Batch mean train Loss:  0.4170	Batch mean val Loss:  0.3037
Batch mean train Loss:  0.4188	Batch mean val Loss:  0.2882
Batch mean train Loss:  0.4124	Batch mean val Loss:  0.2915
Batch mean train Loss:  0.4083	Batch mea

  0%|          | 0/159 [00:00<?, ?it/s]/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:53: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  if ATTACH_DEBUGGER:
/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:19: RuntimeWarning: invalid value encountered in sqrt
  
100%|██████████| 19867/19867 [00:10<00:00, 1948.80it/s]
/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:815: UserWarning: The following variables cannot be serialized: trainer
  warnings.warn(message)


In [1129]:
#!L

# на последок загрузим лучшие веса
@torch.no_grad()
def em():
    embeddings = []
    labels = []
    
    model.load_state_dict(torch.load("best_checkpoint.pth"))
    model.eval()
    
    for batch in tqdm(DataLoader(val_set, shuffle=False)):
        embedding = model.net(batch["sample"].to(DEVICE))
        embeddings.append(embedding.cpu().numpy())
        labels.append(batch["label"].cpu().numpy())
        torch.cuda.empty_cache()
    
    return np.concatenate(embeddings), np.concatenate(labels)

new_embeds, new_labels = em()
new_dists = l2_dist(new_embeds)
top_1_acc, top_5_acc = compute_accs(new_labels, new_dists)

# но лучше особо не стало
print(f"TOP 1 ACCURACY: %.3f" % top_1_acc)
print(f"TOP 5 ACCURACY: %.3f" % top_5_acc)

100%|██████████| 19867/19867 [00:10<00:00, 1929.25it/s]
/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:19: RuntimeWarning: invalid value encountered in sqrt
  


TOP 1 ACCURACY: 0.161
TOP 5 ACCURACY: 0.318


In [ ]:
# your code must be before example

In [1131]:
# for final checkpoint

## Sampler (simple example)

В блоках ниже реализован пример датасета и сэмлера, который возвращает индексы для триплет лосса.

Датасет написан топорно, но основная логика следующая. Если ему на вход приходит `int`, то он возвращает название картинки (`img_name`) и ее лейбл (`img_label`). Если же приходит нечто длиной 3, то он возвращает 3 названия картинок, соответственно. В нашем случае это будет три картинки с двумя одинаковыми лейблами и одним другим: anchor, positive, negative.  
Сэмплер `SimpleTripletSampler`, в свою очередь, отвечает за формирование и поставку в датасет индексов триплетов.

Датасет и сэмлер объединяются внутри даталоадера.

*Hint:* Код написан только лишь для примера, поэтому логика возвращения триплетов может быть неверной.

In [445]:
class SimpleDataset(Dataset):
    def __init__(self, img_names: np.ndarray,
                 img_labels: np.ndarray):
        if len(img_names) != len(img_labels):
            raise ValueError('img_names and img_labels must have equal number of elements')

        self.img_names = img_names
        self.img_labels = img_labels

    def __len__(self):
        return len(self.img_names)
    
    def __getitem__(self, idx):
        if isinstance(idx, int):
            img_name = self.img_names[idx]
            img_label = self.img_labels[idx]
            
            return img_name, img_label
        else:
            assert len(idx) == 3
            
            anc_idx, pos_idx, neg_idx = idx
            anc_img_name = self.img_names[anc_idx]
            pos_img_name = self.img_names[pos_idx]
            neg_img_name = self.img_names[neg_idx]

            return anc_img_name, pos_img_name, neg_img_name


class SimpleTripletSampler(Sampler):
    def __init__(self, dataset: Dataset):
        super().__init__(dataset)

        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __iter__(self):
        for anchor_idx in range(len(self.dataset)):
            positive_idx = self._mine_positive(anchor_idx)
            negative_idx = self._mine_negative(anchor_idx)

            yield anchor_idx, positive_idx, negative_idx

    def _mine_positive(self, anchor_idx: int):
        labels: np.ndarray = self.dataset.img_labels

        anchor_label = labels[anchor_idx]
        pos_idxs = np.nonzero(labels == anchor_label)[0]
        pos_idx = np.random.choice(pos_idxs)

        return pos_idx

    def _mine_negative(self, anchor_idx: int):
        labels: np.ndarray = self.dataset.img_labels

        anchor_label = labels[anchor_idx]
        neg_idxs = np.nonzero(labels != anchor_label)[0]
        neg_idx = np.random.choice(neg_idxs)

        return neg_idx

In [446]:
ex_size = 100
np.random.seed(42)

# в нашем примере названием картинки будет выступать число от 0 до 99, а лейблом число от 0 до 4.
ex_dataset = SimpleDataset(img_names=np.arange(ex_size),
                           img_labels=np.random.randint(0, 5, size=ex_size))
ex_sampler = SimpleTripletSampler(dataset=ex_dataset)

ex_loader = DataLoader(dataset=ex_dataset, batch_size=10, sampler=ex_sampler)

In [447]:
# В этой ячейке мы дергаем первый батч с названиями картинок и достаем их лейблы, 
#  чтобы проверить действительно ли у них одинаковые или разные лейблы.
# Для тренировки сети с триплет лоссом сами лейблы нам не нужны будут.
#  Главное чтобы триплеты картинок формировались правильно: anchor, positive, negative

ex_batch = next(iter(ex_loader))

ex_batch_anc_labels = ex_dataset.img_labels[ex_batch[0]]
ex_batch_pos_labels = ex_dataset.img_labels[ex_batch[1]]
ex_batch_neg_labels = ex_dataset.img_labels[ex_batch[2]]

In [448]:
torch.nonzero()

array([1, 3, 3, 3, 2, 3, 1, 4, 1, 0])

In [449]:
print('All anchor and positive labels are equal:', np.all(ex_batch_anc_labels == ex_batch_pos_labels))
print('Any of anchor and negative labels are equal:', np.any(ex_batch_anc_labels == ex_batch_neg_labels))

All anchor and positive labels are equal: True
Any of anchor and negative labels are equal: False
